# Importing the libraries

In [31]:
import numpy as np
import pandas as pd
import librosa
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# 加載音頻訊號

In [32]:
data_path = '/kaggle/input/recoding-data/UAV Recordings 1s/UAV Recordings 1s'

# 音頻訊號進行MFCCs特徵提取

In [33]:
def load_data(data_dir):
    datasets = []
    labels = []
    categories = os.listdir(data_dir)
    for index, folder in enumerate(categories):
        for filename in os.listdir(data_dir+"/"+folder):
            data, sampling_rate = librosa.load(data_dir+"/"+folder+"/"+filename)
            mfccs_features = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=80)
            mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
            datasets.append(np.array(mfccs_scaled_features))
            labels.append(categories[index])
    return np.array(datasets), np.array(labels)

datasets, labels = load_data(data_path)

In [34]:
datasets = datasets.astype('float32')

labelencoder = LabelEncoder()

labelsOneHot = to_categorical(labelencoder.fit_transform(labels))

# 切分成訓練資料、測試資料，8:2的比例

In [35]:
X_train, X_test, y_train, y_test = train_test_split(datasets, labelsOneHot, test_size = 0.2, stratify = labelsOneHot, random_state = 3)

print("----------Train-------------")
print(X_train.shape)

print("----------Test--------------")
print(X_test.shape)

----------Train-------------
(4512, 80)
----------Test--------------
(1128, 80)


# 類別數量

In [36]:
num_classes = len(np.unique(labels))

# 建立LSTM模型

In [37]:
# 建立一個Sequential模型
model = Sequential()

# 添加一個LSTM層，設定隱藏層的神經元數量為32
model.add(LSTM(128, activation='relu', return_sequences = True, input_shape = (datasets.shape[1], 1)))

model.add(LSTM(64, activation='relu'))

model.add(Dense(64, activation='relu'))

# 使用Dropout防止過度擬合
model.add(Dropout(0.25))

# 輸出層，有num_classes個類別，所以輸出層的輸出維度為num_classes
model.add(Dense(num_classes, activation='softmax'))

# 選擇損失函數、優化方法及成效衡量方式
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 80, 128)        │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 120,453 (470.52 KB)

 Trainable params: 120,453 (470.52 KB)

 Non-trainable params: 0 (0.00 B)

# 模型訓練

In [ ]:
train_history = model.fit(X_train, y_train, validation_split=0.2, batch_size = 100, epochs = 100, verbose = 2)

Epoch 1/100
46/46 - 12s - 267ms/step - accuracy: 0.2463 - loss: 14348.4902 - val_accuracy: 0.1883 - val_loss: 106.2049
Epoch 2/100
46/46 - 8s - 183ms/step - accuracy: 0.2294 - loss: 1483.6798 - val_accuracy: 0.2315 - val_loss: 1327.7349
Epoch 3/100
46/46 - 8s - 177ms/step - accuracy: 0.2167 - loss: 813.2113 - val_accuracy: 0.1705 - val_loss: 150.9724
Epoch 4/100
46/46 - 8s - 175ms/step - accuracy: 0.2095 - loss: 373.4822 - val_accuracy: 0.3045 - val_loss: 88.3031
Epoch 5/100
46/46 - 11s - 232ms/step - accuracy: 0.2147 - loss: 184.2641 - val_accuracy: 0.2658 - val_loss: 51.1464
Epoch 6/100
46/46 - 10s - 216ms/step - accuracy: 0.2145 - loss: 82.6530 - val_accuracy: 0.2248 - val_loss: 20.8499
Epoch 7/100
46/46 - 8s - 177ms/step - accuracy: 0.2291 - loss: 42.0361 - val_accuracy: 0.1783 - val_loss: 31.4125
Epoch 8/100
46/46 - 8s - 179ms/step - accuracy: 0.2291 - loss: 110.8729 - val_accuracy: 0.2802 - val_loss: 47.0433
Epoch 9/100
46/46 - 10s - 228ms/step - accuracy: 0.2729 - loss: 46.0513 

# 評估訓練結果

In [ ]:
def show_train_history(train_history, train, validation):

    plt.plot(train_history.history[train])

    plt.plot(train_history.history[validation])

    plt.title('Train History')

    plt.ylabel('train')

    plt.xlabel('Epoch')

    plt.legend(['train', 'validation'], loc='upper left')

    plt.show()
    

show_train_history(train_history, 'accuracy', 'val_accuracy')
show_train_history(train_history, 'loss', 'val_loss')

# 使用未參與訓練的測試資料測試，模型準確率

In [ ]:
scores = model.evaluate(X_test,y_test,verbose=0)

print('Accuracy = ', scores[1])

# 評估訓練結果

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
confusion_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,7))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()